In [ ]:
import requests
import re
import time
from tqdm import tqdm

API_KEY = "C0C86085E984B6469D40BB790E6B80DF"

def get_steam_id_from_vanity(vanity_url):
    """Resolves a vanity URL to a Steam64 ID."""
    url = f"http://api.steampowered.com/ISteamUser/ResolveVanityURL/v0001/?key={API_KEY}&vanityurl={vanity_url}"
    try:
        response = requests.get(url).json()
        if response.get("response", {}).get("success") == 1:
            return response["response"]["steamid"]
        else:
            print(f"Warning: Could not resolve vanity URL: {vanity_url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error resolving vanity URL {vanity_url}: {e}")
        return None

def get_player_summaries(steam64_id):
    """Fetches player summaries using a Steam64 ID."""
    url = f"https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={API_KEY}&steamids={steam64_id}"
    try:
        response = requests.get(url).json()
        if "response" in response and "players" in response["response"] and len(response["response"]["players"]) > 0:
            return response["response"]["players"][0]
        else:
            print(f"Warning: No player data found for Steam64 ID: {steam64_id}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching player summaries for Steam64 ID {steam64_id}: {e}")
        return None

def get_owned_games(steam64_id):
    """Fetches owned games and playtime using a Steam64 ID."""
    url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={steam64_id}&include_appinfo=1&include_played_free_games=1"
    try:
        response = requests.get(url).json()
        if response.get("response") and response["response"].get("games"):
            return response["response"]["games"]
        else:
            print(f"Warning: No games found or profile is private for Steam64 ID: {steam64_id}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching owned games for Steam64 ID {steam64_id}: {e}")
        return None

def extract_steam_ids_and_full_user_data(base_url, num_pages):
    all_user_data = {}
    all_urls = set()

    try:
        for page_num in tqdm(range(1, num_pages + 1), desc="Processing Pages"):
            url = f"{base_url}?p={page_num}"
            response = requests.get(url)
            response.raise_for_status()
            page_content = response.text

            urls = re.findall(r"(https://steamcommunity\.com/(id|profiles)/([a-zA-Z0-9_-]+|[0-9]+))", page_content)
            all_urls.update(url[0] for url in urls)

        print(f"Collected {len(all_urls)} unique URLs.")

        for url in tqdm(all_urls, desc="Fetching User Info"):
            steam64_id = None
            if "/profiles/" in url:
                steam64_id = url.split("/")[-1]
            elif "/id/" in url:
                vanity_name = url.split("/")[-1]
                steam64_id = get_steam_id_from_vanity(vanity_name)

            if steam64_id:
                player_info = get_player_summaries(steam64_id)
                if player_info:
                    owned_games = get_owned_games(steam64_id)
                    if owned_games:
                        player_info["owned_games"] = owned_games
                    all_user_data[steam64_id] = player_info
                else:
                    print(f"Warning: Failed to retrieve player info for: {url}")

        print("Finished fetching all user info.")
        return all_user_data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

base_url = "https://steamcommunity.com/games/steam/members"
num_pages = 1#339  # Replace with the desired number of pages

all_data = extract_steam_ids_and_full_user_data(base_url, num_pages)

print(f"Total Unique User IDs and URLs: {len(all_data)}")

# Example of how to access the collected user data:
# for steam64_id, user_info in all_data.items():
#     if user_info:
#         print(f"Steam64 ID: {steam64_id}")
#         print(f"  Username: {user_info.get('personaname')}")
#         print(f"  Profile URL: {user_info.get('profileurl')}")
#         print(f"  Avatar: {user_info.get('avatarfull')}")
#         if "owned_games" in user_info:
#             for game in user_info["owned_games"]:
#                 print(f"    Game: {game['name']} (AppID: {game['appid']})")
#                 print(f"    Playtime: {game.get('playtime_forever', 0)} minutes")
#         print("-" * 20)

Processing Pages: 100%|██████████| 1/1 [00:00<00:00,  2.27it/s]


Collected 47 unique URLs.


Fetching User Info:   2%|▏         | 1/47 [00:00<00:13,  3.41it/s]

Error resolving vanity URL nxmxlxss_: Expecting value: line 1 column 1 (char 0)


Fetching User Info:   4%|▍         | 2/47 [00:00<00:11,  3.83it/s]

...

Error resolving vanity URL aaalksndrv: Expecting value: line 1 column 1 (char 0)


Fetching User Info:  98%|█████████▊| 46/47 [00:17<00:00,  2.27it/s]

Error resolving vanity URL Ebpeutop: Expecting value: line 1 column 1 (char 0)


Fetching User Info: 100%|██████████| 47/47 [00:17<00:00,  2.62it/s]

Error resolving vanity URL theaaronhd: Expecting value: line 1 column 1 (char 0)
Finished fetching all user info.
Total Unique User IDs and URLs: 10


In [3]:
# prompt: save all_data in onedrive

import json
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path where you want to save the file in your OneDrive
onedrive_path = '/content/drive/MyDrive/all_data.json'  # Replace with your desired path

# Assuming 'all_data' is your dictionary from the previous code
# Save the data to a JSON file in your OneDrive
with open(onedrive_path, 'w') as f:
    json.dump(all_data, f, indent=4)

print(f"Data saved to: {onedrive_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data saved to: /content/drive/MyDrive/all_data.json
